In [20]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait, Select
import time
import pandas as pd

In [ ]:
# Set up the WebDriver (ensure the path to your WebDriver is correct)
service = Service("/opt/homebrew/bin/chromedriver")
driver = webdriver.Chrome(service=service)

In [ ]:
driver.get("https://data.usaswimming.org/datahub/usas/individualsearch")

# Allow some time for the page to load
time.sleep(5)  # Adjust this as necessary

In [ ]:
# Locate the search input fields by their names or IDs
first_name_input = driver.find_element(By.NAME, "firstOrPreferredName")  # Update with the actual name attribute
last_name_input = driver.find_element(By.NAME, "lastName")

In [ ]:
# Input the swimmer's name
first_name_input.send_keys("Alyssa")  # Example first name
last_name_input.send_keys("Wang")
# Submit the search form
last_name_input.send_keys(Keys.RETURN)

In [6]:
time.sleep(5)

In [ ]:
# First find the row
row = driver.find_element(By.XPATH, "//tr[contains(., 'Rockwood Swim Club')]")
# Then find the button within that row
button = row.find_element(By.CLASS_NAME, "_ActionButton_8nq2x_59")  # Adjust selector as needed
button.click()
time.sleep(5)

In [18]:
wait = WebDriverWait(driver, 10)
dropdown = wait.until(EC.presence_of_element_located((By.NAME, "competitionYearId")))

# Create Select object and select first option
select = Select(dropdown)
select.select_by_value("-1")

In [ ]:
try:
    # Wait for table to be present
    table = wait.until(EC.presence_of_element_located((By.TAG_NAME, "table")))

    # Get headers
    headers = []
    for header in table.find_elements(By.TAG_NAME, "th"):
        headers.append(header.text)

    # Get rows
    rows = []
    for row in table.find_elements(By.TAG_NAME, "tr")[1:]:  # Skip header row
        row_data = []
        for cell in row.find_elements(By.TAG_NAME, "td"):
            row_data.append(cell.text)
        rows.append(row_data)

    # Create DataFrame
    df = pd.DataFrame(rows, columns=headers)
    df = df.rename(columns=lambda x: x.split("\n")[0] if "\n" in x else x)

    # Save to CSV
    df.to_csv("downloaded_table.csv", index=False)
    print("Table downloaded successfully!")

except Exception as e:
    print(f"Error downloading table: {e}")

Table downloaded successfully!
